In [ ]:
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%shell
cp /content/drive/MyDrive/빅데이터프로그래밍/Term_project/* /content

## Prepare

In [ ]:
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver


# Install selenium -- 셀레니움의 너무 잦은 업데이트로 인해 버전 고정 (2023.10)
pip install selenium==4.10

In [ ]:
# 셀레니움 module import
# from selenium import webdriver
################################################################################################################################################################
# 2023.06.08 셀레니움 버전 업데이트로 인해 사용불가
################################################################################################################################################################
# 2023.05.08 selenium webdriver 사용방법 변경
################################################################################################################################################################
# 2023.07.09 셀레니움 버전업으로 인한 코드 변경
################################################################################################################################################################
# 2023.08.02 셀레니움 라이브러리 수정사항 반영
################################################################################################################################################################
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from time import sleep
from bs4 import BeautifulSoup
import requests
import re
import os

# Dataframe
import pandas as pd
df = pd.DataFrame(columns=['idx','star','review'])

# 2023.08.02 셀레니움 라이브러리 수정사항 반영
# https://stackoverflow.com/questions/76428561/typeerror-webdriver-init-got-multiple-values-for-argument-options
service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=service, options=options)


In [ ]:
# 테스트
driver.get("https://www.melon.com/genre/song_list.htm?gnrCode=GN0200&startIndex=200#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&params%5BsteadyYn%5D=N&po=pageObj&startIndex=401")
driver.implicitly_wait(3)
driver.get("https://www.melon.com/song/detail.htm?songId=36963705")
driver.page_source

'<html lang="ko" class="narrow-screen"><head>\n\t\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\n\t<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n\t\n\n\t\n\n\t\n\n\t<title>곡 정보&gt;멜론</title>\n\t<meta name="keywords" content="음악서비스, 멜론차트, 멜론TOP100, 최신음악, 인기가요, 뮤직비디오, 앨범, 플레이어, 스트리밍, 다운로드, 아티스트플러스, 아티스트채널">\n\t<meta name="description" content="No.1 뮤직플랫폼 멜론! 최신 트렌드부터 나를 아는 똑똑한 음악추천까지!">\n\t<meta name="naver-site-verification" content="ee85ff6db1fa8f2226bcb671ecb2bcdbcffb6f8b">\n\t<meta name="google-site-verification" content="q4tbTQhmxa4La3OdNLsNOCxrJ_WV6lUlBFrFW4-HqQc">\n\t<meta property="fb:app_id" content="357952407588971">\n\t<meta property="og:title" content="Breaking the rules - 페리블루 (Ferry Blue)">\n\t<meta property="og:image" content="https://cdnimg.melon.co.kr/cm2/album/images/113/64/412/11364412_20231114104942_500.jpg?5ceb4fd8a684547a92d8ac291d340ad7">\n\t<meta property="og:description" content="음악이 필

## 음원ID, 좋아요 수 얻기

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import re

start_page = "001"
end_page = "025"

genre_codes = {
    # "GN0100": "balad",
    "GN0200": "dance",
    "GN0300": "rap_hiphop",
    "GN0400": "rnb_soul",
    "GN0500": "indie",
    "GN0600": "rock_metal",
}


# 멜론 곡 검색 API URL
def get_recent_song_ids(genre_code="GN0300" ,page_index_range=range(1,2)):

    song_ids = []
    like_counts = []
    for page_index in page_index_range:
        url = f"https://www.melon.com/genre/song_list.htm?gnrCode={genre_code}#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&params%5BsteadyYn%5D=N&po=pageObj&startIndex={page_index*50+1}"
        driver.get(url)
        driver.implicitly_wait(3)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        select_table = soup.select("#frm > div > table > tbody > tr")
        for tr in select_table:
            soup = BeautifulSoup(str(tr), "html.parser")
            href = soup.find(lambda x :x.name=="a" and x.attrs.get("href", "").startswith("javascript:melon.link.goSongDetail(")).attrs['href']
            song_id = re.search(r'\d+', href).group()
            song_ids.append(song_id)
            like = soup.find("span", class_="cnt").get_text(strip=True).lstrip("총건수").replace(",", "")
            like_counts.append(like)
    return song_ids, like_counts

def save_to_csv(genre, recent_songs_likes):
    # with open(f"/content/{genre}_0033_051_melonIds.txt", 'w', newline='', encoding='utf-8-sig') as txt:
    df = pd.DataFrame({"songId": recent_songs_likes[0], "like_count": recent_songs_likes[1]})
    print(df)
    df.to_csv(f'/content/{genre}_001_025_melonIds.csv', index=False)



for gnrcode in genre_codes.keys():
    recent_song_ids = get_recent_song_ids(gnrcode, range(int(start_page), int(end_page)+1))
    save_to_csv(genre_codes[gnrcode], recent_song_ids)



        songId like_count
0     36963705         48
1     36965084          3
2     36964255          0
3     36962675        130
4     36961184         18
...        ...        ...
1245  35985167     195253
1246  35984500        334
1247  35980350         74
1248  35962512       5747
1249  35976650       1670

[1250 rows x 2 columns]
        songId like_count
0     36963705         48
1     36965084          3
2     36964255          0
3     36962675        130
4     36961184         18
...        ...        ...
1245  35985167     195253
1246  35984500        334
1247  35980350         74
1248  35962512       5747
1249  35976650       1670

[1250 rows x 2 columns]
        songId like_count
0     36963705         48
1     36965084          3
2     36964255          0
3     36962675        130
4     36961184         18
...        ...        ...
1245  35985167     195253
1246  35984500        334
1247  35980350         74
1248  35962512       5747
1249  35976650       1670

[1250 rows x 

## 나머지 요소 크롤링

### 준비

In [ ]:
start_page = "001"
end_page = "025"
genre = "rock_metal"

In [ ]:
songs_dict = {
    'Title': [],
    'Singer': [],
    'Song_ID': [],
    'Release_Date': [],
    'Like_Count': [],
    'lyrics': [],
    'Album': [],
    'Genre': [],
}

def foo(songId, like=None):
    headers = {
    "Host": "www.melon.com",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "save-data": "on",
    "Referer":"https://fqhf3hvcdzt-496ff2e9c6d22116-0-colab.googleusercontent.com/"
    }

    url = f"https://www.melon.com/song/detail.htm?songId={songId}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        driver.get(url)
        driver.implicitly_wait(3)
        response = driver.page_source
    else:
        response = response.content
    soup = BeautifulSoup(response, 'html.parser')
    title = soup.select_one('.song_name').get_text(strip=True).lstrip("곡명")
    singer = soup.select_one('div.artist > a > span:nth-child(1)').get_text(strip=True)
    release_date = soup.select_one('div.meta > dl > dd:nth-child(4)').get_text(strip=True)
    album = soup.select_one('div.meta > dl > dd:nth-child(2) > a').get_text(strip=True)
    genre = soup.find(lambda x: x.name=="dt" and x.text=="장르").find_next_sibling().get_text(strip=True)
    lyrics_soup = soup.select_one('#d_video_summary')
    try:
        lyrics = lyrics_soup.get_text('\n', strip=True)
    except:
        lyrics = None
    return (title, singer, songId, release_date, like, lyrics, album, genre)



In [ ]:
import concurrent.futures
import pandas as pd
import multiprocessing
import numpy as np

genre = "rap_hiphop"
df = pd.read_csv(f"/content/{genre}_001_025_melonIds.csv", encoding="utf-8-sig",) #index_col="Unnamed: 0")

### No Multiprocessing

In [ ]:
# 기존 DataFrame을 songs_dict에 추가
start_range = 1100
for i in range(start_range, len(df)):
    songs_dict = {
        'Title': [],
        'Singer': [],
        'Song_ID': [],
        'Release_Date': [],
        'Like_Count': [],
        'lyrics': [],
        'Album': [],
        'Genre': [],
    }
    for key, val in zip(songs_dict.keys(), foo(df.iloc[i].songId, df.iloc[i].like_count)):
        songs_dict[key].append(val)

    # songs_dict를 DataFrame으로 변환하여 CSV 파일에 추가
    temp_df = pd.DataFrame(songs_dict)
    if i == start_range:
        temp_df.to_csv(f"/content/{genre}_{start_page}_{end_page}_melonIds_result.csv", mode='w', index=False, encoding="utf-8-sig")
    else:
        temp_df.to_csv(f"/content/{genre}_{start_page}_{end_page}_melonIds_result.csv", mode='a', header=False, index=False, encoding="utf-8-sig")

### Multithreading

In [ ]:
import concurrent.futures
import pandas as pd
import multiprocessing
import numpy as np

df = pd.read_csv(f"/content/{genre}_001_025_melonIds.csv", encoding="utf-8-sig",)

def process_data(row):
    songs_dict = {
        'Title': [],
        'Singer': [],
        'Song_ID': [],
        'Release_Date': [],
        'Like_Count': [],
        'lyrics': [],
        'Album': [],
    }
    for key, val in zip(songs_dict.keys(), foo(row.songId, row.like_count)):
        songs_dict[key].append(val)

    return pd.DataFrame(songs_dict)

def main():
    start_range = 501
    num_cores = multiprocessing.cpu_count()  # 사용 가능한 CPU 스레드 수
    print(num_cores)

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_cores) as executor:
        df_split = np.array_split(df.iloc[start_range:], num_cores)
        results = executor.map(process_data, df_split)

    output_file = f"/content/{genre}_{start_page}_{end_page}_melonIds_result.csv"
    for i, result in enumerate(results):
        mode = 'w' if i == start_range else 'a'
        header = i == start_range
        result.to_csv(output_file, mode=mode, header=header, index=False, encoding="utf-8-sig")

if __name__ == '__main__':
    main()


2


AttributeError: ignored

### Multiprocessing(미완성)

In [ ]:
import numpy as np
from multiprocessing import Pool

def parallelize_dataframe(df, func, num_cores):
    df_split = np.array_split(df, num_cores)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    print(df.iloc[0])
    pool.close()
    pool.join()
    return df

def process_data(df):
    songs_dict = {
        'Title': [],
        'Singer': [],
        'Song_ID': [],
        'Release_Date': [],
        'Like_Count': [],
        'lyrics': [],
        'Album': [],
    }
    for row in df.itertuples(index=False):
        for key, val in zip(songs_dict.keys(), foo(row.songId, row.like_count)):
            songs_dict[key].append(val)

    return pd.DataFrame(songs_dict)
start_range = 1024
num_cores = multiprocessing.cpu_count()  # 사용 가능한 CPU 코어 수
print(num_cores)
result_df = parallelize_dataframe(df, process_data, num_cores)


2


Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 365, in get
    res = self._reader.recv_bytes()
  File "<ipython-input-27-851

KeyboardInterrupt: ignored

  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


# etc


In [ ]:
for genre in genre_codes.values():
    print(genre)
    df = pd.read_csv(f"/content/{genre}_001_025_melonIds.csv", encoding="utf-8-sig")
    df.drop("Unnamed: 0", axis=1).to_csv(f"/content/{genre}_001_025_melonIds.csv", index=False, encoding="utf-8-sig")
    # pd.read_csv("/content/{genre}_001_025_melonIds.csv", encoding="utf-8-sig", index=False)

dance


KeyError: ignored

In [ ]:
df = pd.read_csv("/content/{genre}_001_025_melonIds.csv", encoding="utf-8-sig")
df

,songId,like_count
0,36963705,48
1,36965084,3
2,36964255,0
3,36962675,130
4,36961184,18
...,...,...
1245,35985167,195248
1246,35984500,334
1247,35980350,74
1248,35962512,5745


In [ ]:
songId = 36963705
headers = {
    "Host": "www.melon.com",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "save-data": "on",
    "Referer":"https://fqhf3hvcdzt-496ff2e9c6d22116-0-colab.googleusercontent.com/"
    }

url = f"https://www.melon.com/song/detail.htm?songId={songId}"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
genre = soup.find(lambda x: x.name=="dt" and x.text=="장르").find_next_sibling().get_text(strip=True)
genre

'댄스, 인디음악'

In [ ]:
lyrics_soup.get_text('\n').strip()

"끝이 보이지 않는 어둠 속에\n한 줄기 빛이 되어준 너\n헤어나올 수 없는 Like a Black hole\n침식 되어 가는 나\n텅 빈 나의 눈 속에 니가 들어와\n눈 부시게 가득 채워\n푸른빛 물든 설렘의 색깔은 Blue\n다시 빛 나도록\n따라와 다시 한 번 꽃을 피워줄게\n파도 위에 올라타봐\n다 알아 내 매력에 빠져버린 너를\nYou wanna be my love forever babe\nBreaking the rules\nforget the button, to stop me\nCome out, in front of me\nBreaking the rules\nforget the button, to stop me\nCome out, in front of me\n숨겨 놓았던 내 속에 Wonder land\n살며시 보여주고 싶어\n중력 같았던 널 향한 이끌림\n손을 뻗어 만져봐\nYeah, Yeah!\n날 가뒀던 선들의 모순\n날 눌렀던 악몽의 굴레\n지겨워 벗어날래 이젠\n거침없이 널 펼쳐봐\n우리란 바다에 닥친 거센 바람에\n지지 않을래 더 큰 파랑을 일 차례\n두려워할 필요 없어 널 그대로 보여 줘\nDon't be Fake\n따라와 다시 한 번 꽃을 피워줄게\n파도 위에 올라타봐\n다 알아 내 매력에 빠져버린 너를\nYou wanna be my love forever babe\nBreaking the rules\nforget the button, to stop me\nCome out, in front of me\nBreaking the rules\nforget the button, to stop me\nCome out, in front of me\n감춰 두었던 부끄러웠던 내 모습\n이젠 보내줄게\n두 손에 꽉쥐어진 빛바랜 후회들은\n이제 인정해 Game over\nYeah Yeah Yeah\n따라와 다시 한 번 꽃을 피워줄게\n파도 위에 올라타봐\n다 알아 내 매력에 빠져버린 너를\nYou wanna be my love forever babe\nB

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import re

# 장르 번호
index=1
sess = requests.Session()
params = {}
headers = {
    "Host": "www.melon.com",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "save-data": "on",
    "Cookie": "__T_=1; PCID=17002250845185087861648; PC_PCID=17002250845185087861648; __T_=1; POC=MP10"
}
# 멜론 곡 검색 API URL
def get_latest_song_ids(sess=requests.Session(), headers=str(), page_index=4):
    if not headers:
        headers = {
            "Host": "www.melon.com",
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
            "save-data": "on",
            "Cookie": "__T_=1; PCID=17002250845185087861648; PC_PCID=17002250845185087861648; __T_=1; POC=MP10"
        }

    url = f"https://www.melon.com/genre/song_list.htm?gnrCode=GN0200"
    res = sess.get(url, headers=headers)

    url = f"https://www.melon.com/genre/song_list.htm?gnrCode=GN0200#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&params%5BsteadyYn%5D=N&po=pageObj&startIndex=201"
    res = sess.get(url, headers=headers, allow_redirects=True)
    if res.status_code != 200:
        raise Exception("no!")
    soup = BeautifulSoup(res.content, "html.parser")
    select_table = soup.select("#frm > div > table > tbody > tr")
    song_ids = []
    for tr in select_table:
        soup = BeautifulSoup(str(tr), "html.parser")
        href = soup.find(lambda x :x.name=="a" and x.attrs.get("href", "").startswith("javascript:melon.link.goSongDetail(")).attrs['href']
        song_id = re.search(r'\d+', href).group()
        song_ids.append(song_id)
    return song_ids
get_latest_song_ids(page_index=5)

In [ ]:
url = f"https://www.melon.com/genre/song_list.htm?gnrCode=GN0200#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&params%5BsteadyYn%5D=N&po=pageObj&startIndex=201"
res = sess.get(url, headers=headers)
BeautifulSoup(res.content, "html.parser")

In [ ]:
from bs4 import BeautifulSoup

# 여기에 HTML 코드를 넣으세요.

# BeautifulSoup을 사용하여 HTML 파싱
soup = BeautifulSoup(res.content, 'html.parser')

# BeautifulSoup에서 XPath를 사용하는 함수 정의
def find_element_by_xpath(soup, xpath):
    elements = soup.find_all(lambda tag: tag.name == '*' and tag.xpath(xpath))
    return elements

# 사용할 XPath
xpath_to_find = 'tbody'

# XPath를 사용하여 요소 찾기
found_elements = find_element_by_xpath(soup, xpath_to_find)

# # 선택된 요소 확인
# for element in found_elements:
#     print(element.text.strip())  # 요소 텍스트 출력
found_elements

[]

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# 장르 번호
genre="1"

# 멜론 곡 검색 API URL
melon_search_url = f"https://www.melon.com/genre/song_listPaging.htm?startIndex=101&pageSize=50&gnrCode=GN0200&dtlGnrCode=&orderBy=NEW&steadyYn=N0"

# 곡 검색
melon_search_response = requests.get(melon_search_url.format(query=release_date))
melon_search_soup = BeautifulSoup(melon_search_response.text, "html.parser")

# 곡 목록
song_list = melon_search_soup.find("ul", class_="lst_song").find_all("li")

# 곡 가사
song_data = []
for song in song_list:
    # 곡 정보
    song_title = song.find("a", class_="link_song").text
    song_artist = song.find("span", class_="artist").text

    # 곡 가사 URL
    song_lyrics_url = song.find("a", class_="link_lyrics").get("href")

    # 곡 가사 요청
    song_lyrics_response = requests.get(song_lyrics_url)
    song_lyrics_soup = BeautifulSoup(song_lyrics_response.text, "html.parser")

    # 곡 가사
    song_lyrics = song_lyrics_soup.find("div", class_="lyrics_wrap").text

    # 곡 데이터 저장
    song_data.append([song_title, song_artist, song_lyrics])

# 엑셀파일로 저장
with open("melon_lyrics.csv", "w", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["곡 제목", "가수", "가사"])
    for row in song_data:
        writer.writerow(row)


AttributeError: ignored

In [ ]:
melon_search_url = f"https://www.melon.com/genre/song_list.htm?gnrCode=GN0{genre}00"
genre=1

NameError: ignored

In [ ]:
def get_music_info_by_songId(songId):
    x_paths = {
    'title': '//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[1]',
    'singer': '//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[2]/a/span[1]',
    'release_date': '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]',
    'songId': None,
    'like_count': '//*[@id="d_like_count"]',
    'genre': '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]',
    'lyrics': '//*[@id="d_video_summary"]',
    'album': '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[1]/a',
    }
    url = f"https://www.melon.com/song/detail.htm?songId={songId}"
    driver.get(url)
    driver.implicitly_wait(3)
    # title = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[1]').text.strip()
    # singer = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[2]/a/span[1]').get_attribute("innerText").strip()
    # release_date = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').get_attribute("innerText").strip()
    # like_count = driver.find_element(By.XPATH, '//*[@id="d_like_count"]').get_attribute("innerText").strip()
    # album = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[1]/a').get_attribute("innerText").strip()
    # genre = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]').get_attribute("innerText").strip()
    result_list = []
    for key in x_paths.keys():
        try:
            if key == 'title':
                result_list.append(driver.find_element(By.XPATH, x_paths[key]).text.strip())
                continue
            if key == 'songId':
                result_list.append(songId)
                continue
            result_list.append(driver.find_element(By.XPATH, x_paths[key]).get_attribute("innerText").strip())
        except:
            result_list.append(None)
    return result_list


def convert_music_info_to_csv(filepath):
    songIds = []
    songs_dict = {
        'Title': [],
        'Singer': [],
        'Song_ID': [],
        'Release_Date': [],
        'Like_Count': [],
        'lyrics': [],
        'Album': [],
    }
    with open(filepath, 'r', encoding="utf-8-sig") as txt:
        songIds = txt.readlines()[250:350]
    for songId in songIds:
        try:
            infos = get_music_info_by_songId(songId.strip())
            print(infos)[:-1]
        except Exception as e:
            print(e)
            # driver.quit()
            # driver = webdriver.Chrome(service=service, options=options)
            infos = get_music_info_by_songId(songId.strip())
        for key, info in zip(songs_dict.keys(), infos):
            songs_dict[key].append(info)
    df = pd.DataFrame(songs_dict)
    df.to_csv(f"{filepath.rstrip('.txt')}.csv", index=False, encoding='utf-8-sig')